In [1]:
# import libraries
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8

2025-02-11 12:07:46.312629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739272066.349865 3958020 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739272066.362223 3958020 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 12:07:46.446849: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("./data/SWaT_Dataset_Attack_v0.csv")

df.columns = [
	'TIMESTAMP','FIT101','LIT101','MV101','P101','P102','AIT201','AIT202','AIT203','FIT201','MV201','P201','P202','P203',
    'P204','P205','P206','DPIT301','FIT301','LIT301','MV301','MV302','MV303','MV304','P301','P302','AIT401','AIT402','FIT401',
    'LIT401','P401','P402','P403','P404','UV401','AIT501','AIT502','AIT503','AIT504','FIT501','FIT502','FIT503','FIT504',
    'P501','P502','PIT501','PIT502','PIT503','FIT601','P601','P602','P603','OUTCOME'
]

# The first row only contains labels
# 'TIMESTAMP' is irrelevant for the thesis
# The other dropped columns contain either only 0s, only 1s or only 2s and are therefore irrelevant

df = df.iloc[1:]
df = df.drop(['TIMESTAMP', 'P202', 'P301', 'P401', 'P404', 'P502', 'P601', 'P603'], axis = 1)

# The dataset labels attacks by 'A ttack' and 'Attack', and labels normal data as 'Normal'
# To keep the same structure in all datasets, the 'A ttack' and 'Attack' values are changed to '-1' and normal values to '1'

df['OUTCOME'].replace(to_replace = ['A ttack', 'Attack'], value = 1, inplace = True)
df['OUTCOME'].replace(to_replace = ['Normal'], value = 0, inplace = True)

# data types need to be numeric to be encoded to z-scores --> convert column object data types to numerics

cols = df.columns[df.columns != 'OUTCOME']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# Encoding the feature vectors to z-scores

cols = list(df.columns[df.columns != 'OUTCOME'])
for col in cols:
    df[col] = ((df[col] - df[col].mean())/df[col].std(ddof=0))
    
display(df)

/tmp/ipykernel_3958020/4193474766.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['OUTCOME'].replace(to_replace = ['A ttack', 'Attack'], value = 1, inplace = True)
/tmp/ipykernel_3958020/4193474766.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['OUTCOME'].replace(to_replace = ['Norm

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT502,FIT503,FIT504,P501,PIT501,PIT502,PIT503,FIT601,P602,OUTCOME
1,0.614181,-0.671446,0.693862,0.665191,-0.083632,1.471050,-1.150246,0.501006,0.656906,0.655352,...,0.307535,0.294030,0.296326,0.282974,0.302691,1.542312,0.315175,-0.102994,-0.095828,0
2,0.650194,-0.671760,0.693862,0.665191,-0.083632,1.471050,-1.166990,0.501006,0.654185,0.655352,...,0.296729,0.294030,0.306616,0.282974,0.302950,1.542312,0.315175,-0.102994,-0.095828,0
3,0.688088,-0.670819,0.693862,0.665191,-0.083632,1.471050,-1.166990,0.501006,0.654185,0.655352,...,0.258144,0.294030,0.306616,0.282974,0.302950,1.542312,0.313825,-0.102994,-0.095828,0
4,0.717382,-0.666747,0.693862,0.665191,-0.083632,1.471050,-1.166990,0.501006,0.654866,0.655352,...,0.258144,0.294030,0.306616,0.282974,0.302950,1.542312,0.311464,-0.102994,-0.095828,0
5,0.750975,-0.663615,0.693862,0.665191,-0.083632,1.471050,-1.166990,0.501006,0.655772,0.655352,...,0.232678,0.294030,0.306616,0.282974,0.300874,1.542312,0.311464,-0.102994,-0.095828,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449914,0.709588,-0.698073,0.693862,0.665191,-0.083632,-1.200282,0.959110,-1.105095,0.669378,0.655352,...,0.281680,0.292003,0.281287,0.282974,0.307359,-0.359522,0.301340,-0.103797,-0.095828,0
449915,0.700450,-0.691181,0.693862,0.665191,-0.083632,-1.200282,0.959110,-1.105095,0.669378,0.655352,...,0.232678,0.292003,0.281287,0.282974,0.306062,-0.359522,0.299316,-0.103797,-0.095828,0
449916,0.685669,-0.688989,0.693862,0.665191,-0.083632,-1.200282,0.959110,-1.105095,0.669945,0.655352,...,0.223033,0.292003,0.281287,0.282974,0.306062,-0.359522,0.299316,-0.103797,-0.095828,0
449917,0.677068,-0.688675,0.693862,0.665191,-0.083632,-1.200282,0.959110,-1.105095,0.669945,0.655352,...,0.206055,0.292003,0.298701,0.282974,0.306062,-0.359522,0.299316,-0.103797,-0.095828,0


In [ ]:
from sklearn.feature_selection import mutual_info_classif

# 🔹 Definisci feature e target
X = df.drop(columns=["OUTCOME"])  # Tutte le colonne tranne il target
y = df["OUTCOME"]  # Target binario (0 = normale, 1 = anomalia)

# 🔹 Calcola la Mutual Information
mi_scores = mutual_info_classif(X, y)

# 🔹 Crea un DataFrame con i risultati
mi_results = pd.DataFrame({"Feature": X.columns, "MI_Score": mi_scores})

# 🔹 Filtra solo le feature con MI > 0 (cioè, che portano informazione)
threshold = 0.2098
selected_features = mi_results[mi_results["MI_Score"] > threshold]

# 🔹 Ordina in base all'importanza
selected_features = selected_features.sort_values(by="MI_Score", ascending=False)

# 🔹 Stampa le feature selezionate
print("Feature più informative rispetto alle anomalie:")
print(selected_features)

# 🔹 Se vuoi solo i nomi delle feature per usarle in MATLAB
important_feature_names = selected_features["Feature"].tolist()
print("\nFeature selezionate:", important_feature_names)


# Creating normal and attack masks
normal_mask = df[df.OUTCOME == 0]
attack_mask = df[df.OUTCOME == 1]

Feature più informative rispetto alle anomalie:
   Feature  MI_Score
5   AIT201  0.240373
30  AIT501  0.220037
29   UV401  0.210289
38    P501  0.210171

Feature selezionate: ['AIT201', 'AIT501', 'UV401', 'P501']


In [5]:
# Supponiamo che `df` sia il tuo DataFrame originale
selected_features = ['AIT201', 'AIT501', 'UV401', 'P501']

# Creiamo un nuovo DataFrame con solo queste feature + il target
df_selected = df[selected_features + ["OUTCOME"]]

# Salviamo il file CSV
df_selected.to_csv("./data/relevant_features.csv", index=False)

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import scipy.io as sio  # Per salvare in .mat (MATLAB format)

# 🔹 Carica il dataset (se non è già in memoria)
df = pd.read_csv("./data/relevant_features.csv")

# 🔹 Seleziona solo le feature di input (senza OUTCOME)
X = df.drop(columns=["OUTCOME"])
y = df["OUTCOME"]

# 🔹 Filtra solo dati normali per training e validation set
df_normal = df[df["OUTCOME"] == 0]
df_anomaly = df[df["OUTCOME"] == 1]

# 🔹 Prendi i primi 9000 dati normali per il training set
X_train = df_normal.iloc[:9000, :-1].values  # Escludi OUTCOME
y_train = df_normal.iloc[:9000, -1].values  # Target

# 🔹 Prendi i successivi 9000 dati normali per il validation set
X_val = df_normal.iloc[9000:18000, :-1].values
y_val = df_normal.iloc[9000:18000, -1].values

# 🔹 Crea test set bilanciato (50% anomalie) - 1750 normali + 1750 anomalie
X_test_balanced = pd.concat([df_normal.iloc[18000:19750, :-1], df_anomaly.iloc[:1750, :-1]]).values
y_test_balanced = pd.concat([df_normal.iloc[18000:19750, -1], df_anomaly.iloc[:1750, -1]]).values

# 🔹 Crea test set con 5% anomalie - 3325 normali + 175 anomalie
X_test_5perc = pd.concat([df_normal.iloc[19750:23075, :-1], df_anomaly.iloc[1750:1925, :-1]]).values
y_test_5perc = pd.concat([df_normal.iloc[19750:23075, -1], df_anomaly.iloc[1750:1925, -1]]).values

# 🔹 Normalizza con MinMaxScaler (range [0,1])
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test_balanced = scaler.transform(X_test_balanced)
X_test_5perc = scaler.transform(X_test_5perc)

# 🔹 Salva i dataset in formato MATLAB (.mat) con array
sio.savemat("./data/fuzzy_data.mat", {
    "X_train": X_train, 
    "y_train": y_train, 
    "X_val": X_val, 
    "y_val": y_val, 
    "X_test_balanced": X_test_balanced, 
    "y_test_balanced": y_test_balanced, 
    "X_test_5perc": X_test_5perc, 
    "y_test_5perc": y_test_5perc
})

In [ ]:
# Creating X_training and X_testing datasets
X_train, X_test = train_test_split(df, test_size = 0.2, random_state = 42)

X_train = X_train[X_train.OUTCOME == 0]
X_train = X_train.drop(['OUTCOME'], axis=1)

y_test = X_test['OUTCOME']
X_test = X_test.drop(['OUTCOME'], axis=1)

X_train = X_train.values
X_test = X_test.values

print(X_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Building the autoencoder model and showing its summary
ae = model = Sequential()
model.add(Dense(35, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(35, activation='relu'))
model.add(Dense(X_train.shape[1]))

ae.summary()

In [ ]:
# Train the model
nb_epoch = 50
batch_size = 64

ae.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath="./model.SWAT.keras", verbose=0)

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

history = ae.fit(X_train, X_train, epochs=nb_epoch, batch_size=batch_size, shuffle=True,
                 validation_data=(X_test, X_test), verbose=1, callbacks=[checkpointer, tensorboard]).history

In [7]:
# Save the model 
ae = load_model('model.SWAT.keras')

In [ ]:
# Plot model loss over the amount of epochs
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right');
# plt.savefig('loss_SWAT_GoodModel.png')

In [ ]:
# Plotting model accuracy over the amount of epochs
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right');
# plt.savefig('accuracy_SWaT_GoodModel.png')

In [ ]:
# Calculating the predictions of the autoencoder model on X_test testing sample
predictions = ae.predict(X_test)

In [50]:
# Calculating MSE and reconstruction error
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test})

In [ ]:
error_df.describe()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
normal_error_df = error_df[(error_df['true_class'] == 0) & (error_df['reconstruction_error'] < 0.1)]
_ = ax.hist(normal_error_df.reconstruction_error.values, bins=10)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
fraud_error_df = error_df[error_df['true_class'] == 1]
_ = ax.hist(fraud_error_df.reconstruction_error.values, bins=10)

In [54]:
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

In [ ]:
# Calculating and plotting the ROC curve
fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([-1,1],[-1,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
# plt.savefig('ROC_SWAT2.png')
plt.show()

In [ ]:
# Plotting Recall vs Precision plot
precision, recall, th = precision_recall_curve(error_df.true_class, error_df.reconstruction_error)
plt.plot(recall, precision, 'b', label='Precision-Recall curve')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()
# high precision = low false positive
# high recall = low false negative 

In [ ]:
# Plotting precision metrics for different threshold values to find optimal threshold
plt.plot(th, precision[1:], 'b', label='Threshold-Precision curve')
plt.title('Precision for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Precision')
# plt.savefig('precision_threshold_SWAT.png')
plt.show()

In [ ]:
# Plotting recall metrics for different threshold values to find optimal threshold
plt.plot(th, recall[1:], 'b', label='Threshold-Recall curve')
plt.title('Recall for different threshold values')
plt.xlabel('Reconstruction error')
plt.ylabel('Recall')
plt.show()

In [59]:
# Setting the manual threshold
threshold = 0.03

In [ ]:
# Plot reconstruction error per datapoint index for the manually defined threshold
groups = error_df.groupby('true_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, group.reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Attack" if name == 1 else "Normal")
ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();
# plt.savefig('Threshold SWaT.png')

In [ ]:
# Plotting the confusion matrix
threshold = 0.01
LABELS = ["Normal", "Attack"]
y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
conf_matrix = confusion_matrix(error_df.true_class, y_pred)

plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()